# Lectura 33: Lazy API - Plan y ejecución de una consulta

## Plan de consulta

Para cualquier consulta lazy, Polars tiene ambos:

- **Un plan no optimizado:** este contiene el conjunto de pasos en código qu ele proporcionamos.
- **Un plan optimizado:** que contiene cambios realizados por el optimizador de consultas

Consideremos la siguiente consulta:

In [ ]:
import polars as pl
from polars import col

query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .with_columns(tiempo_aire_hrs=(col('AIR_TIME') / 60))
    .filter(col('MONTH') > 7)
)

Primero visualizaremos el plan no optimizado con la función `show_graph` estableciendo el parámetro `optimized=False`.

**Nota Importante:**

Para visualizar los gráficos será necesario instalar la librería `graphviz` la cual la podemos instalar ejecutando en una celda de código el comando `pip install graphviz`. Además, necesitará instalar el programa `graphviz` en su sistema operativo.

**Para errores y dudas pueden visitar el siguiente enlace:**

(Solución de errores graphviz)[https://stackoverflow.com/questions/35064304/runtimeerror-make-sure-the-graphviz-executables-are-on-your-systems-path-aft]

In [ ]:
# Descomentar y ejecutar solo si no está instalada la librería graphviz
# !pip3 install graphviz

In [ ]:
query.show_graph(optimized=False)

La visualización del plan de consulta debe leerse de abajo hacia arriba. Expliquemos como leer esta salida:

- Cada rectángulo corresponde a una etapa del plan de consulta
- El `sigma` significa SELECCIÓN e indica cualquier condición de filtro
- El `pi` significa PROYECCIÓN e indica la elección de un subconjunto de columnas

Ahora visualizaremos el pan de consulta optimizado.

In [ ]:
query.show_graph()

### Imprimir el plan de consulta

También podemos imprimir el plan no optimizado con `explain(optimized=False)`.

In [ ]:
query.explain(optimized=False)

Imprimamos el plan de consulta optimizado.

In [ ]:
query.explain()

## Ejecución de una consulta

In [ ]:
query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .with_columns(tiempo_aire_hrs=(col('AIR_TIME') / 60))
    .filter(col('MONTH') > 7)
)

Si tuviéramos que ejecutar el código anterior en el dataset de vuelos, la consulta no se evaluaría. En cambio, Polars toma cada línea de código, la agrega al gráfico de consulta interno y lo optimiza.

Cuando ejecutamos el código, Polars ejecuta el gráfico de consulta optimizado de forma predeterminada.

Podemos ejecutar nuestra consulta en el conjunto de datos completo llamando al método `.collect` en la consulta.

In [ ]:
query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .with_columns(tiempo_aire_hrs=(col('AIR_TIME') / 60))
    .filter(col('MONTH') > 7)
    .collect()
)

query

Con el método por defecto collect, Polars procesa todos sus datos como un solo lote. Esto significa que todos los datos deben caber en la memoria disponible en el punto de mayor uso de la memoria en la consulta.

## Ejecución en datos más grandes que la memoria

Si nuestros datos requieren más memoria de la que tenemos disponible, es posible que Polars pueda procesar los datos en lotes  (batches en inglés) utilizando el modo de streaming. Para usar el modo streaming simplemente pasamos el argumento `streaming=True` a `collect`.

In [ ]:
query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .with_columns(tiempo_aire_hrs=(col('AIR_TIME') / 60))
    .filter(col('MONTH') > 7)
    .collect(streaming=True)
)

query

## Ejecución en un conjunto de datos parcial

Mientras escribimos, optimizamos o verificamos nuestra consulta en un conjunto de datos grande, consultar todos los datos disponibles puede provocar un proceso de desarrollo lento.

En su lugar, podemos ejecutar la consulta con el método `.fetch`. El método `.fetch` toma un parámetro `n_rows` e intenta "obtener" (`fetch`) esa cantidad de filas en la fuente de datos. Sin embargo, no se puede garantizar el número de filas, ya que la API lazy no cuenta cuántas filas hay en cada etapa de la consulta.

Veamos un ejemplo:

In [ ]:
query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .filter(col('AIR_TIME') > 100)
)

query.fetch(n_rows=20)